In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import easyocr
import cv2

In [24]:
DATADIR="img"
CATEGORIES=["ID_CARD","OTHERS"]
IMG_SIZE=100
training_data=[]

def create_training_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

create_training_data()

In [25]:
print(len(training_data))

949


In [27]:
import random
random.shuffle(training_data)

In [28]:
x=[]
y=[]
for features, label in training_data:
    x.append(features)
    y.append(label)
x=np.array(x).reshape(-1,IMG_SIZE, IMG_SIZE,3)

In [29]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle

In [30]:
x=x/255.0

y=np.array(y)

In [31]:
model=Sequential()
model.add(Conv2D(64, (3,3), input_shape=x.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss="binary_crossentropy", 
              optimizer="adam", 
              metrics=['accuracy'])
model.fit(x, y, batch_size=6, epochs=24, validation_split=0.3)

Epoch 1/24
111/111 [==============================] - 9s 73ms/step - loss: 0.8178 - accuracy: 0.6792 - val_loss: 0.3956 - val_accuracy: 0.8316
Epoch 2/24
111/111 [==============================] - 8s 71ms/step - loss: 0.2255 - accuracy: 0.9142 - val_loss: 0.1848 - val_accuracy: 0.9404
Epoch 3/24
111/111 [==============================] - 8s 76ms/step - loss: 0.1245 - accuracy: 0.9578 - val_loss: 0.2371 - val_accuracy: 0.9263
Epoch 4/24
111/111 [==============================] - 9s 78ms/step - loss: 0.0809 - accuracy: 0.9684 - val_loss: 0.2251 - val_accuracy: 0.9579
Epoch 5/24
111/111 [==============================] - 8s 75ms/step - loss: 0.0343 - accuracy: 0.9895 - val_loss: 0.2151 - val_accuracy: 0.9614
Epoch 6/24
111/111 [==============================] - 9s 77ms/step - loss: 0.0360 - accuracy: 0.9880 - val_loss: 0.2199 - val_accuracy: 0.9439
Epoch 7/24
111/111 [==============================] - 9s 81ms/step - loss: 0.0182 - accuracy: 0.9925 - val_loss: 0.2672 - val_accuracy: 0.9544

In [32]:
CATEGORIES=["ID_CARD","OTHERS"]
def prepare(filepath):
    IMG_SIZE=100
    img_array=cv2.imread(filepath)
    new_array=cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
prediction=model.predict([prepare("TEST_ID_CARD/test2.jpeg")])
print(CATEGORIES[int(prediction[0][0])])

1/1 [==============================] - 0s 78ms/step
ID_CARD


In [33]:
from joblib import Parallel, delayed
import joblib

joblib.dump(model, 'my_model.pkl')

['my_model.pkl']